In [ ]:
#default_exp core

In [ ]:
#export
from fastcore.utils import *
from fastcore.foundation import *
from fastcore.meta import *

import pprint,inspect,json,copy,urllib,mimetypes
from inspect import signature,Parameter,Signature
from urllib.parse import urlencode
from ghapi.metadata import funcs
from urllib.request import Request,urlretrieve

In [ ]:
#export
GH_HOST = "https://api.github.com"
_DOC_URL = 'https://docs.github.com/'

In [ ]:
#export
def _mk_param(nm, **kwargs): return Parameter(nm, kind=Parameter.POSITIONAL_OR_KEYWORD, **kwargs)

def mk_sig(req_args, def_args):
    params =  [_mk_param(k) for k in req_args]
    params += [_mk_param(k, default=v) for k,v in def_args.items()]
    return Signature(params)

In [ ]:
#export
class GhVerb:
    __slots__ = 'path,verb,tag,name,summary,url,route_ps,params,data,hdrs,__doc__'.split(',')
    def __init__(self, path, verb, oper, summary, url, params, data, hdrs, kwargs):
        tag,name = oper.split('/')
        name = name.replace('-','_')
        path,route_ps,_ = partial_format(path, **kwargs)
        __doc__ = summary
        store_attr()
    
    def __call__(self, *args, headers=None, **kwargs):
        headers = {**headers,**self.hdrs} if headers else self.hdrs
        flds = [o for o in self.route_ps+self.params+self.data if o not in kwargs]
        for a,b in zip(args,flds): kwargs[b]=a
        route_p,query_p,data_p = [{p:kwargs[p] for p in o if p in kwargs}
                                 for o in (self.route_ps,self.params,self.data)]
        return dict2obj(urlsend(GH_HOST+self.path, self.verb, headers=headers,
                              route=route_p, query=query_p, data=data_p))

    @property
    def __signature__(self): return mk_sig(self.route_ps, dict.fromkeys(self.params+self.data))
    __call__.__signature__ = __signature__

    def _repr_markdown_(self):
        params = ', '.join(self.route_ps+self.params+self.data)
        return f"[{self.tag}/{self.name}]({_DOC_URL}{self.url})({params}): *{self.summary}*"
    __repr__ = _repr_markdown_

class _GhVerbGroup:
#     def __init__(self, verbs): super().__init__(**{o.name:o for o in verbs})
    def __init__(self, verbs):
        for o in verbs: setattr(self, o.name, o)
    def _repr_markdown_(self): return "\n".join(f'- [{k}]({_DOC_URL}{v.url})' for k,v in self.items())

In [ ]:
#export
_docroot = 'https://docs.github.com/en/free-pro-team@latest/rest/reference/'

In [ ]:
#export
class GhApi:
    def __init__(self, owner=None, repo=None, token=None, **kwargs):
        self.headers = { 'Accept': 'application/vnd.github.v3+json' }
        if token: self.headers['Authorization'] = 'token ' + token
        if owner: kwargs['owner'] = owner
        if repo:  kwargs['repo' ] = repo
        funcs_ = L(funcs).starmap(GhVerb, hdrs=self.headers, kwargs=kwargs)
        self._fs = {k:_GhVerbGroup(v) for k,v in groupby(funcs_, 'tag').items()}

    def __dir__(self): return super().__dir__() + list(self._fs)
    def _repr_markdown_(self): return "\n".join(f'- [{o}]({_docroot+o})' for o in self._fs)
    def __getattr__(self,k): return self._fs[k] if k in self._fs else stop(AttributeError(k))

### Operation groups

In [ ]:
#hide
github_token = os.environ['FASTRELEASE_TOKEN']

In [ ]:
api = GhApi('fastai', 'ghapi-test', token=github_token)

The following groups of endpoints are provided, which you can list at any time along with a link to documentation for all endpoints in that group, by displaying the `GhApi` object:

In [ ]:
api

- [apps](https://docs.github.com/en/free-pro-team@latest/rest/reference/apps)
- [oauth-authorizations](https://docs.github.com/en/free-pro-team@latest/rest/reference/oauth-authorizations)
- [codes-of-conduct](https://docs.github.com/en/free-pro-team@latest/rest/reference/codes-of-conduct)
- [emojis](https://docs.github.com/en/free-pro-team@latest/rest/reference/emojis)
- [enterprise-admin](https://docs.github.com/en/free-pro-team@latest/rest/reference/enterprise-admin)
- [billing](https://docs.github.com/en/free-pro-team@latest/rest/reference/billing)
- [activity](https://docs.github.com/en/free-pro-team@latest/rest/reference/activity)
- [gists](https://docs.github.com/en/free-pro-team@latest/rest/reference/gists)
- [gitignore](https://docs.github.com/en/free-pro-team@latest/rest/reference/gitignore)
- [issues](https://docs.github.com/en/free-pro-team@latest/rest/reference/issues)
- [licenses](https://docs.github.com/en/free-pro-team@latest/rest/reference/licenses)
- [markdown](https://docs.github.com/en/free-pro-team@latest/rest/reference/markdown)
- [meta](https://docs.github.com/en/free-pro-team@latest/rest/reference/meta)
- [orgs](https://docs.github.com/en/free-pro-team@latest/rest/reference/orgs)
- [actions](https://docs.github.com/en/free-pro-team@latest/rest/reference/actions)
- [interactions](https://docs.github.com/en/free-pro-team@latest/rest/reference/interactions)
- [migrations](https://docs.github.com/en/free-pro-team@latest/rest/reference/migrations)
- [projects](https://docs.github.com/en/free-pro-team@latest/rest/reference/projects)
- [repos](https://docs.github.com/en/free-pro-team@latest/rest/reference/repos)
- [teams](https://docs.github.com/en/free-pro-team@latest/rest/reference/teams)
- [reactions](https://docs.github.com/en/free-pro-team@latest/rest/reference/reactions)
- [rate-limit](https://docs.github.com/en/free-pro-team@latest/rest/reference/rate-limit)
- [checks](https://docs.github.com/en/free-pro-team@latest/rest/reference/checks)
- [code-scanning](https://docs.github.com/en/free-pro-team@latest/rest/reference/code-scanning)
- [git](https://docs.github.com/en/free-pro-team@latest/rest/reference/git)
- [pulls](https://docs.github.com/en/free-pro-team@latest/rest/reference/pulls)
- [scim](https://docs.github.com/en/free-pro-team@latest/rest/reference/scim)
- [search](https://docs.github.com/en/free-pro-team@latest/rest/reference/search)
- [users](https://docs.github.com/en/free-pro-team@latest/rest/reference/users)

## Calling endpoints

The GitHub API's endpoint names generally start with a verb like "get", "list", "delete", "create", etc, followed `_`, then by a noun such as "ref", "webhook", "issue", etc. Each endpoint has a different signature, which you can see by using <kbd>Shift</kbd>-<kbd>Tab</kbd> in Jupyter, or just viewing the endpoint object, which also provides a link to the official GitHub documentation:

In [ ]:
api.git.get_ref

[git/get_ref](https://docs.github.com/rest/reference/git#get-a-reference)(ref): *Get a reference*

They are called using standard Python method syntax:

In [ ]:
ref = api.git.get_ref('heads/master')
test_eq(ref.object.type, 'commit')

Information about the endpoint are available as attributes:

In [ ]:
api.git.get_ref.path,api.git.get_ref.verb

('/repos/fastai/ghapi-test/git/ref/{ref}', 'get')

You can get a list of all endpoints available in a group, along with a link to documentation for each, by viewing the group:

In [ ]:
api.git

AttributeError: '_GhVerbGroup' object has no attribute 'items'

For "list" endpoints, the noun will be a plural form, e.g.:

In [ ]:
#hide
for hook in api.repos.list_webhooks(): api.repos.delete_webhook(hook.id)

In [ ]:
hooks = api.repos.list_webhooks()
test_eq(len(hooks), 0)

You can pass dicts, lists, etc. directly, where they are required for GitHub API endpoints:

In [ ]:
url = 'https://example.com'
cfg = dict(url=url, content_type='json', secret='XXX')
hook = api.repos.create_webhook(config=cfg, events=['ping'])
test_eq(hook.config.url, url)

Let's confirm that our new webhook has been created:

In [ ]:
hooks = api.repos.list_webhooks()
test_eq(len(hooks), 1)
test_eq(hooks[0].events, ['ping'])

Finally, we can delete our new webhook:

In [ ]:
api.repos.delete_webhook(hooks[0].id)

### Convenience methods

There are some multi-step processes in the GitHub API that `GhApi` provide convenient wrappers for. The methods currently available are shown below; do not hesitate to [create an issue](https://github.com/fastai/ghapi-test/issues) or pull request if there are other processes that you'd like to see supported better.

In [ ]:
@patch
def _upload_file(self:GhApi, url:str, fn:str):
    "Upload `fn` to endpoint `url`"
    mime = mimetypes.guess_type(fn, False)[0] or 'application/octet-stream'
    headers = {**api.headers, 'Content-Type':mime}
    data = Path(fn).read_bytes()
    return urlsend(url, 'POST', headers=headers, query = {'name':fn}, data=data)

In [ ]:
@patch
def create_release(self:GhApi, tag_name, branch='master', name=None, body='',
                   draft=False, prerelease=False, files=None):
    "Wrapper for `api.repos.create_release` which also uploads `files`"
    if name is None: name = 'v'+tag_name
    rel = api.repos.create_release(tag_name, target_commitish=branch, name=name, body=body,
                                   draft=draft, prerelease=prerelease)
    url = rel.upload_url.replace('{?name,label}','')
    for file in listify(files): self._upload_file(url, file)
    return rel

Creating a release and attaching files to it is normally a multi-stage process, so `create_release` wraps this up for you. It takes the same arguments as [`api.repos.create_release`](https://docs.github.com/rest/reference/repos#create-a-release), along with `files`, which can contain a single file name, or a list of file names to upload to your release:

In [ ]:
rel = api.create_release('0.0.1', files=['README.md'])
test_eq(rel.name, 'v0.0.1')
rels = api.repos.list_releases()
test_eq(len(rels), 1)

We can check that our file has been uploaded; GitHub refers to them as "assets":

In [ ]:
assets = api.repos.list_release_assets(rels[0].id)
test_eq(assets[0].name, 'README.md')
test_eq(assets[0].content_type, 'text/markdown')

In [ ]:
@patch
def delete_release(self:GhApi, release):
    "Delete a release and its associated tag"
    ref = f'tags/{release.tag_name}'
    tag = api.git.list_matching_refs(ref)
    api.repos.delete_release(rel.id)
    api.git.delete_ref(ref)

We can delete our release and confirm that it is removed:

In [ ]:
api.delete_release(rels[0])
test_eq(len(api.repos.list_releases()), 0)

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 90_build_lib.ipynb.
Converted index.ipynb.
